In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import librosa.display
import soundfile as sound
import os

In [5]:
class_list = ['baby','bicycle','boiling','car','carpassing','clock','dog','door','fire','glass','jackhammer','kettle','scream','siren','speech','unknown','whistle']

for class_name in class_list:
    ThisPath = 'path/' + class_name + '/'
    OutPath = 'path/'
    file_list = os.listdir(ThisPath)
    
    cnt = 0
    for file_name in file_list:
        
        #it's same number with door sound
        if(cnt>475):
            continue
    
        #read audio
        audio_path = ThisPath + file_name
        y, sr = sound.read(audio_path)
        
        #stero to mono
        if(y[0].shape == (2,)):
            new_y = []
            new_y = (y[:,0] + y[:,1]) / 2
            y = new_y
        
        #resample (ex : 44100 -> 16000)
        y = librosa.resample(y, sr, 16000)
        duration = y.shape[0]/16000
        
        #duration
        if(duration < 0.4):
            continue

        seconds_division = 50
        window_length = int(16000/seconds_division)
    
        y_abs = np.absolute(y)
        y_mean = y_abs.mean()
    
        normalized = []
        ratio = 0.3
        
        print (duration*seconds_division)
        for i in range(int(duration*seconds_division)):
            if i % 1000 == 0:
                print (i)
            window = y[i * window_length : (i+1) * window_length]
            window_abs = np.absolute(window)
            if(window_abs.mean() > y_mean*ratio):
                normalized = np.concatenate((normalized,window))
        
        s = 6400 # if 6400 --> 0.4s
        print(file_name, "sr : ", 16000, "duration : ", duration, "normalized_duration", normalized.shape[0]/16000)        
        for i in range(int(normalized.shape[0]/s)):
            cnt = cnt+1
            if(cnt>475):
                continue
            sound.write(OutPath + class_name + str(cnt) + '.wav', normalized[i*s:(i+1)*s],16000)
        

959.2750000000001
0
eval_unknown.wav sr :  16000 duration :  19.1855 normalized_duration 19.18


In [3]:
import re

OutPath = 'path'
ResultPath = 'path (.csv)'
result_file_list = os.listdir(OutPath)
class_list = ['baby','bicycle','boiling','car','carpassing','clock','dog','door','fire','glass','jackhammer','kettle','scream','siren','speech','unknown','whistle']

cnt = 0
p = re.compile("[^0-9]")
with open(ResultPath, 'w') as f:
    for i in range(len(result_file_list)):
        if i == 0:
            f.write('filenames'+',')
            f.write('scene_label')
            f.write('\n')
        tmp = "".join(p.findall(result_file_list[i])).replace('.wav','').replace('_nomobile','')
        if tmp in class_list:
            class_name = tmp
        f.write(result_file_list[i]+','+class_name)
        f.write('\n')
        cnt += 1